In [ ]:
# 🚨 STOCK NEWS NOTIFICATION through SMS my code
import requests
from twilio.rest import Client
from datetime import datetime, timedelta
import sys
sys.stdin.reconfigure(encoding='utf-8') # fix the UnicodeEncodeError because news can be written in many languages
sys.stdout.reconfigure(encoding='utf-8')

STOCK = "TSLA"
COMPANY_NAME = "Tesla Inc"

STOCK_ENDPOINT = "https://www.alphavantage.co/query"
NEWS_ENDPOINT = "https://newsapi.org/v2/everything"

account_sid = "ACddae********bfaab501a697d7b9e9d"
auth_token = "1cb1353d*****************29853"
client = Client(account_sid, auth_token)

# Get the stock information

parameters_stock = {
    "function": "TIME_SERIES_INTRADAY",
    "symbol": STOCK,
    "interval": "60min",
    "apikey": "WGMFOBN******5WM",
}

response = requests.get(url=STOCK_ENDPOINT, params=parameters_stock)
response.raise_for_status()
stock_price = response.json()

time_yesterday = datetime.strftime(datetime.now() - timedelta(2),'%Y-%m-%d')
time_bf_yesterday = datetime.strftime(datetime.now() - timedelta(3),'%Y-%m-%d')
time_now = datetime.now().hour

close_yesterday = float(stock_price["Time Series (60min)"][f"{time_yesterday} 20:00:00"]["4. close"])
close_bf_yesterday = float(stock_price["Time Series (60min)"][f"{time_bf_yesterday} 20:00:00"]["4. close"])

price_difference = round(abs(close_bf_yesterday - close_yesterday), 3)

percentage_difference = round(abs(close_yesterday / close_bf_yesterday)*100-100, 0)

# Get the news

parameters_com = {
    "apiKey": "3589f24***************fc98",
    "q": "tesla",
    "from": "2022-07-26",
    "sortBy": "publishedAt",
    }

response = requests.get(url=NEWS_ENDPOINT, params=parameters_com)
response.raise_for_status()
com_news = response.json()

text = ""
for i in range(0, 3):
    a = com_news["articles"][i]["title"]
    b = com_news["articles"][i]["url"]
    text += f"Headlines: {a}.\n"
    text += f"Link: {b}.\n"

if percentage_difference >= 0:
    message_content = f"TSLA 🔺{percentage_difference}%\n" + text
else:
    message_content = f"TSLA 🔻{percentage_difference}%\n" + text

message = client.messages \
.create(
     body=message_content,
     from_="+17432008088",
     to="+849*******5"
 )
print(message.status)

In [ ]:
# 🚨 STOCK NEWS NOTIFICATION through SMS answer
import requests
from twilio.rest import Client

VIRTUAL_TWILIO_NUMBER = "your virtual twilio number"
VERIFIED_NUMBER = "your own phone number verified with Twilio"

STOCK_NAME = "TSLA"
COMPANY_NAME = "Tesla Inc"

STOCK_ENDPOINT = "https://www.alphavantage.co/query"
NEWS_ENDPOINT = "https://newsapi.org/v2/everything"

STOCK_API_KEY = "YOUR OWN API KEY FROM ALPHAVANTAGE"
NEWS_API_KEY = "YOUR OWN API KEY FROM NEWSAPI"
TWILIO_SID = "YOUR TWILIO ACCOUNT SID"
TWILIO_AUTH_TOKEN = "YOUR TWILIO AUTH TOKEN"

## STEP 1: Use https://www.alphavantage.co/documentation/#daily
# When stock price increase/decreases by 5% between yesterday and the day before yesterday then print("Get News").

#Get yesterday's closing stock price
stock_params = {
    "function": "TIME_SERIES_DAILY",
    "symbol": STOCK_NAME,
    "apikey": STOCK_API_KEY,
}

response = requests.get(STOCK_ENDPOINT, params=stock_params)
data = response.json()["Time Series (Daily)"]
data_list = [value for (key, value) in data.items()]
yesterday_data = data_list[0]
yesterday_closing_price = yesterday_data["4. close"]
print(yesterday_closing_price)

#Get the day before yesterday's closing stock price
day_before_yesterday_data = data_list[1]
day_before_yesterday_closing_price = day_before_yesterday_data["4. close"]
print(day_before_yesterday_closing_price)

#Find the positive difference between 1 and 2. e.g. 40 - 20 = -20, but the positive difference is 20. Hint: https://www.w3schools.com/python/ref_func_abs.asp
difference = float(yesterday_closing_price) - float(day_before_yesterday_closing_price)
up_down = None
if difference > 0:
    up_down = "🔺"
else:
    up_down = "🔻"

#Work out the percentage difference in price between closing price yesterday and closing price the day before yesterday.
diff_percent = round((difference / float(yesterday_closing_price)) * 100)
print(diff_percent)

    ## STEP 2: Instead of printing ("Get News"), actually get the first 3 news pieces for the COMPANY_NAME.

#Instead of printing ("Get News"), use the News API to get articles related to the COMPANY_NAME.
#If difference percentage is greater than 5 then print("Get News").
if abs(diff_percent) > 1:
    news_params = {
        "apiKey": NEWS_API_KEY,
        "qInTitle": COMPANY_NAME,
    }

    news_response = requests.get(NEWS_ENDPOINT, params=news_params)
    articles = news_response.json()["articles"]

    #Use Python slice operator to create a list that contains the first 3 articles. Hint: https://stackoverflow.com/questions/509211/understanding-slice-notation
    three_articles = articles[:3]
    print(three_articles)

    ## STEP 3: Use Twilio to send a seperate message with each article's title and description to your phone number.

    #Create a new list of the first 3 article's headline and description using list comprehension.
    formatted_articles = [f"{STOCK_NAME}: {up_down}{diff_percent}%\nHeadline: {article['title']}. \nBrief: {article['description']}" for article in three_articles]
    print(formatted_articles)
    #Send each article as a separate message via Twilio.
    client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)

    #TODO 8. - Send each article as a separate message via Twilio.
    for article in formatted_articles:
        message = client.messages.create(
            body=article,
            from_=VIRTUAL_TWILIO_NUMBER,
            to=VERIFIED_NUMBER
        )

In [ ]:
# 🚨 Habit Tracking using Pixela
import requests
from datetime import datetime, timedelta

# graph link https://pixe.la/v1/users/thane/graphs/graph1.html

pixela_endpoint = "https://pixe.la/v1/users"
TOKEN = "iuadhwugugaw"
USER_NAME = "thane"

user_params = {
    "token": TOKEN,
    "username": USER_NAME,
    "agreeTermsOfService": "yes",
    "notMinor": "yes",
}

# response = requests.post(url=pixela_endpoint, json=user_params)
# print(response.text)

graph_endpoint = f"{pixela_endpoint}/{USER_NAME}/graphs"

graph_config = {
    "id": "graph1",
    "name": "Code Time",
    "Unit": "minutes",
    "type": "int",
    "color": "shibafu",
}

header = {
    "X-USER-TOKEN": TOKEN,
}

# response = requests.post(url=graph_endpoint, json=graph_config, headers=header)
# print(response.text)

time_now = datetime.now()

node_endpoint = f"{pixela_endpoint}/thane/graphs/graph1" #/{time_now.strftime('%Y%m%d')}

node_data = {
    "date": time_now.strftime('%Y%m%d'),
    "quantity": input("How many minutes you spent on your code excercise?: "),
}

response = requests.post(url=node_endpoint, json=node_data, headers=header)
# post is to add a pixel for the first time and get is to get the information of a pixel.
# post = /v1/users/<username>/graphs/<graphID>.
# put is to update a pixel and delete is to delete it.
# put = delete = get = /v1/users/<username>/graphs/<graphID>/<yyyyMMdd>. 
print(response.text)

In [ ]:
# 🚨 Workout Tracking with google sheet
from datetime import datetime
from requests.auth import HTTPBasicAuth
import requests

basic = HTTPBasicAuth('Thane******', 'niue/da******dh')

API_KEY = "7661d3ad8b8ba*************98b4"
API_ID = "ff6***86"

headers = {
    "x-app-id": API_ID,
    "x-app-key": API_KEY,
}

parameters = {
#  "query":input("Which exercise you did?: "),
 "gender":"male",
 "weight_kg":60,
 "height_cm":174,
 "age":19
}

# response = requests.get(url="https://trackapi.nutritionix.com/v2/natural/exercise", params=parameters, headers=headers)
# response.raise_for_status() #The url had been broken so the code cannot run.
# data = response.json()

sheety_get_post_url = "https://api.sheety.co/5187d5e2d94173524a084b3d76bb6cc9/thane'sWorkout/workouts"
sheety_put_delete_url = "https://api.sheety.co/5187d5e2d94173524a084b3d76bb6cc9/thane'sWorkout/workouts/[Object ID]"

time_now = datetime.now()

row = {
    "workout": {
        "date": time_now.strftime('%Y%m%d'),
        "time": time_now.strftime('%H:%M:%S'),
        "exercise": "Running",
        "duration": "30",
        "calories": "167",
    }
}

header = {
    "Authorization": "Basic VGhhbmU0NTYzODpu********QmF3ZGg=",
}

response = requests.get(url=sheety_get_post_url, auth=basic, headers=header)
response.raise_for_status()
data = requests.post(sheety_get_post_url, json=row) # Add data to the workouts sheet.

sheet_response = requests.post(
    sheety_get_post_url, 
    json=row, 
    auth=basic,
    headers=header
)

response = requests.get(url=sheety_get_post_url, auth=basic, headers=header)
print(response.text)

In [ ]:
import os

# How to set environment variables.
APP_ID = os.environ["APP_ID"]
API_KEY = os.environ["API_KEY"]

# How to get an environment variable.
APP_ID = os.environ.get("APP_ID")
API_KEY = os.environ.get("API_KEY")

# Create a .env file to save environment variables.

In [ ]:
# 🚨 FLIGHT DEAL project main
from flight_search import FlightSearch
from flight_data import FlightData
from data_manager import DataManager 
from notification_manager import NotificationManager

Noti = NotificationManager()

cityFrom = "SGN"
cityTo = input("Where you want to go to? ")

flight_info = FlightData()
parameters = flight_info.flight_info(cityFrom, cityTo)

sheet = DataManager()
sheet_data = sheet.get_data()

flight_price_data = FlightSearch()
flight_price = flight_price_data.get_price(parameters)

for data in sheet_data:
    if cityTo == data["iataCode"]:
        if min(flight_price) < data["lowerPrice"]:
            sheet_data = sheet.update_price(cityTo, min(flight_price))
            message_to = Noti.message_body(data["city"], min(flight_price))

In [ ]:
# 🚨 FLIGHT CLUB project main
# Due to the expiration of the sheety payment, the project is still in progress. :<
# You can read the end code in this file. My own code will be added later.